# Problem-1: Ant Colony Optimization(ACO)

In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# ============================================================
#  ACO FOR 1D BIN PACKING
# ============================================================

class ACOSingleDimBinPacking:
    """
    Ant Colony Optimization for 1D bin packing.

    We keep pheromone values for (item, bin_index) choices:
        tau[i, b] = attractiveness of putting item i into bin b.

    Items are first sorted in decreasing order to encourage tight packings.
    Each ant builds a solution sequentially and decides for each item whether
    to use an existing feasible bin or open a new one.
    """

    def __init__(
        self,
        item_sizes,
        capacity,
        num_ants=20,
        max_iters=100,
        alpha=1.0,
        beta=4.0,
        rho=0.1,
        Q=1.0,
        greedy_prob=0.1,
        seed=0,
    ):
        self.rng = np.random.default_rng(seed)

        # Problem data
        self.original_sizes = np.array(item_sizes, dtype=float)
        self.capacity = float(capacity)
        self.n_items = len(self.original_sizes)

        # Sort items in decreasing size to improve tightness
        self.sorted_idx = np.argsort(-self.original_sizes)
        self.sizes = self.original_sizes[self.sorted_idx]

        # Theoretical upper bound: each item in its own bin
        self.max_bins = self.n_items

        # ACO parameters
        self.num_ants = num_ants
        self.max_iters = max_iters
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q
        self.greedy_prob = greedy_prob

        # Pheromone matrix: item x potential bin-slot
        self.tau = np.ones((self.n_items, self.max_bins), dtype=float)

        # Best solution tracking
        self.best_num_bins = self.n_items
        self.best_unused = float("inf")
        self.best_assignment = None  # bin index per sorted item
        self.best_loads = None

        # Convergence history
        self.best_history = []     # best-so-far #bins
        self.iter_best_history = []  # best-per-iteration #bins

    # --------------------------------------------------------

    def _build_solution(self):
        """
        Construct one solution (one ant):

        Returns:
            assignment : np.array, len n_items, bin index for each item
            loads      : list of bin loads for used bins
        """
        assignment = -np.ones(self.n_items, dtype=int)
        loads = []
        used_bins = 0

        for i in range(self.n_items):
            size_i = self.sizes[i]

            cand_bins = []
            desirability = []

            # Existing bins (0 .. used_bins-1)
            for b in range(used_bins):
                remaining = self.capacity - loads[b]
                if remaining >= size_i:
                    # Heuristic: prefer tighter fits -> 1 / remaining
                    eta = 1.0 / (remaining + 1e-9)
                    pher = self.tau[i, b]
                    score = (pher ** self.alpha) * (eta ** self.beta)
                    cand_bins.append(b)
                    desirability.append(score)

            # Option to open a new bin: use index `used_bins`
            if used_bins < self.max_bins:
                remaining_new = self.capacity - size_i
                eta_new = 1.0 / (remaining_new + 1e-9)
                pher_new = self.tau[i, used_bins]
                score_new = (pher_new ** self.alpha) * (eta_new ** self.beta)
                cand_bins.append(used_bins)
                desirability.append(score_new)

            desirability = np.array(desirability, dtype=float)

            # Handle degenerate case
            if desirability.sum() <= 0:
                probs = np.full_like(desirability, 1.0 / len(desirability))
            else:
                probs = desirability / desirability.sum()

            # Greedy vs roulette selection
            if self.rng.random() < self.greedy_prob:
                chosen_idx = int(np.argmax(probs))
            else:
                chosen_idx = self.rng.choice(len(cand_bins), p=probs)

            chosen_bin = cand_bins[chosen_idx]
            assignment[i] = chosen_bin

            if chosen_bin == used_bins:
                # Opening a new bin
                loads.append(size_i)
                used_bins += 1
            else:
                loads[chosen_bin] += size_i

        return assignment, loads

    # --------------------------------------------------------

    def _simple_local_search(self, assignment, loads, max_passes=2):
        """
        Very small local search:

        - Try to move items from later bins into earlier bins if capacity allows.
        - Goal is to reduce the number of used bins and improve pack tightness.

        This is intentionally light so runtime stays reasonable.
        """
        assignment = assignment.copy()
        loads = list(loads)
        used_bins = len(loads)

        for _ in range(max_passes):
            improved = False

            # Recompute item lists per bin
            bin_items = [[] for _ in range(used_bins)]
            for i, b in enumerate(assignment):
                if 0 <= b < used_bins:
                    bin_items[b].append(i)

            # Try to empty bins from the end
            for b in reversed(range(used_bins)):
                items = bin_items[b]
                if not items:
                    continue

                # Check if we can move all items from bin b to earlier bins
                feasible = True
                move_plan = []
                for i in items:
                    size_i = self.sizes[i]
                    placed = False
                    for t in range(b):
                        if loads[t] + size_i <= self.capacity:
                            move_plan.append((i, t))
                            placed = True
                            break
                    if not placed:
                        feasible = False
                        break

                if not feasible:
                    continue

                # Commit the moves
                for i, t in move_plan:
                    old_bin = assignment[i]
                    loads[old_bin] -= self.sizes[i]
                    loads[t] += self.sizes[i]
                    assignment[i] = t

                # Now bin b might be empty
                improved = True

            # Compact bins: remove empty ones and renumber
            if improved:
                new_index = {}
                new_loads = []
                idx = 0
                for b in range(used_bins):
                    if loads[b] > 1e-9:
                        new_index[b] = idx
                        new_loads.append(loads[b])
                        idx += 1

                for i in range(len(assignment)):
                    b = assignment[i]
                    if b in new_index:
                        assignment[i] = new_index[b]
                    else:
                        assignment[i] = -1  # should not be used

                loads = new_loads
                used_bins = len(loads)
            else:
                break

        return assignment, loads

    # --------------------------------------------------------

    def _evaluate(self, loads):
        """Compute (#bins, total unused capacity)."""
        used_loads = [l for l in loads if l > 1e-9]
        num_bins = len(used_loads)
        unused = sum(self.capacity - l for l in used_loads)
        return num_bins, unused

    # --------------------------------------------------------

    def run(self):
        """
        Run the ACO metaheuristic and store the best solution found.
        Returns:
            best_num_bins, best_unused_capacity, runtime_seconds, best_history_list
        """
        start = time.time()

        for it in range(self.max_iters):
            iter_best_bins = self.n_items
            iter_best_unused = float("inf")
            iter_best_assign = None
            iter_best_loads = None

            # Each ant builds a solution (no local search yet)
            for _ in range(self.num_ants):
                assignment, loads = self._build_solution()
                num_bins, unused = self._evaluate(loads)

                if (num_bins < iter_best_bins) or (
                    num_bins == iter_best_bins and unused < iter_best_unused
                ):
                    iter_best_bins = num_bins
                    iter_best_unused = unused
                    iter_best_assign = assignment
                    iter_best_loads = loads

            # Local improvement only on best of this iteration
            iter_best_assign, iter_best_loads = self._simple_local_search(
                iter_best_assign, iter_best_loads, max_passes=2
            )
            iter_best_bins, iter_best_unused = self._evaluate(iter_best_loads)

            # Global pheromone update
            self.tau *= (1.0 - self.rho)  # evaporation

            deposit = self.Q / (iter_best_bins + 1e-9)
            used_bins = len(iter_best_loads)
            for i, b in enumerate(iter_best_assign):
                if 0 <= b < used_bins:
                    self.tau[i, b] += deposit

            # Update global best
            if (iter_best_bins < self.best_num_bins) or (
                iter_best_bins == self.best_num_bins and iter_best_unused < self.best_unused
            ):
                self.best_num_bins = iter_best_bins
                self.best_unused = iter_best_unused
                self.best_assignment = iter_best_assign.copy()
                self.best_loads = list(iter_best_loads)

            self.iter_best_history.append(iter_best_bins)
            self.best_history.append(self.best_num_bins)

        runtime = time.time() - start
        return self.best_num_bins, self.best_unused, runtime, self.best_history

    # --------------------------------------------------------

    def plot_convergence(self, filepath):
        plt.figure()
        plt.plot(self.best_history, label="Best so far")
        plt.plot(self.iter_best_history, label="Iteration best", alpha=0.6)
        plt.xlabel("Iteration")
        plt.ylabel("Number of bins")
        plt.title("ACO convergence (1D)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(filepath)
        plt.close()

    def plot_load_distribution(self, filepath):
        if self.best_assignment is None:
            return

        used_bins = self.best_num_bins
        loads = np.zeros(used_bins)
        for i, b in enumerate(self.best_assignment):
            if 0 <= b < used_bins:
                loads[b] += self.sizes[i]

        loads_sorted = np.sort(loads)

        plt.figure()
        plt.bar(np.arange(len(loads_sorted)), loads_sorted)
        plt.axhline(self.capacity, linestyle="--", label="Capacity")
        plt.xlabel("Bin index (sorted)")
        plt.ylabel("Load")
        plt.title("Bin load distribution (1D)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(filepath)
        plt.close()


# ============================================================
#  SIMPLE OR-LIB 1D LOADER
# ============================================================

def load_orlib_1d(filepath, instance_index=0, float_items=False):
    """
    Load one 1D bin packing instance from Beasley's OR-Library file format.

    File structure:
        P
        name_1
        C n B
        sizes...
        name_2
        ...

    Returns:
        sizes (list), capacity (int/float), best_known (int), name (str)
    """
    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    pos = 0
    num_instances = int(lines[pos])
    pos += 1

    if instance_index >= num_instances:
        raise ValueError("Instance index out of range")

    for inst in range(num_instances):
        name = lines[pos]
        pos += 1
        tokens = lines[pos].split()
        pos += 1

        if float_items:
            capacity = float(tokens[0])
        else:
            capacity = int(tokens[0])

        n_items = int(tokens[1])
        best_known = int(tokens[2])

        sizes = []
        for _ in range(n_items):
            if float_items:
                sizes.append(float(lines[pos]))
            else:
                sizes.append(int(lines[pos]))
            pos += 1

        if inst == instance_index:
            return sizes, capacity, best_known, name

    # Should not reach here
    raise RuntimeError("Instance not found after parsing file.")


# ============================================================
#  2D AREA-BASED APPROXIMATION USING THE 1D SOLVER
# ============================================================

class AreaApprox2DBinPacking:
    """
    Very simple 2D bin packing approximation:

    - For each rectangular item, use area = width * height.
    - For bins, define capacity_area = max(width * height) over all bins.
    - Apply the 1D ACO solver to these areas as if it was a 1D instance.

    This does not ensure geometric feasibility, it's an area-based heuristic,
    but it lets us reuse the 1D ACO to get an indicative packing quality.
    """

    def __init__(
        self,
        item_dims,
        bin_dims,
        num_ants=15,
        max_iters=80,
        alpha=1.0,
        beta=3.0,
        rho=0.1,
        Q=1.0,
        seed=0,
    ):
        self.item_dims = item_dims
        self.bin_dims = bin_dims
        self.num_bins_avail = len(bin_dims)

        # Convert to 1D areas
        self.item_areas = [w * h for (w, h) in item_dims]
        self.bin_areas = [w * h for (w, h) in bin_dims]
        capacity_area = max(self.bin_areas)

        # Reuse 1D ACO on areas
        self.solver = ACOSingleDimBinPacking(
            self.item_areas,
            capacity_area,
            num_ants=num_ants,
            max_iters=max_iters,
            alpha=alpha,
            beta=beta,
            rho=rho,
            Q=Q,
            seed=seed,
        )

    def run(self):
        best_bins, unused_area, runtime, conv = self.solver.run()
        return best_bins, unused_area, runtime, conv

    def plot_convergence(self, filepath):
        self.solver.plot_convergence(filepath)

    def plot_load_distribution(self, filepath):
        # Distribution of used bin areas (approximation)
        self.solver.plot_load_distribution(filepath)


# ------------------------------------------------------------

def load_orlib_2d(filepath):
    """
    Parse OR-Library 2D bin packing file (binpack2d).

    This is a simplified parser that assumes the following structure
    (which matches the published format):

        P
        name_1
        n_1 B_1 [best1]
        (bin dims lines...)
        (item dims lines...)
        name_2
        ...

    Bin lines may be:
        W H
        W H count

    Item lines: one "w h" per item.

    Returns:
        list of dicts:
            {
              "name": str,
              "n_items": int,
              "bin_dims": list[(W,H)],
              "item_dims": list[(w,h)],
              "best_known": int or None
            }
    """
    instances = []

    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    pos = 0
    num_instances = int(lines[pos])
    pos += 1

    for _ in range(num_instances):
        name = lines[pos]
        pos += 1

        header_tokens = lines[pos].split()
        pos += 1

        n_items = int(header_tokens[0])
        num_bins = int(header_tokens[1])
        best_known = int(header_tokens[2]) if len(header_tokens) > 2 else None

        # Read bin dimensions until we have num_bins entries
        bin_dims = []
        while len(bin_dims) < num_bins:
            tokens = lines[pos].split()
            pos += 1
            if len(tokens) == 2:
                w, h = map(int, tokens)
                bin_dims.append((w, h))
            elif len(tokens) == 3:
                w, h, cnt = map(int, tokens)
                for _ in range(cnt):
                    bin_dims.append((w, h))
                    if len(bin_dims) == num_bins:
                        break
            else:
                # Unexpected line; skip
                continue

        # Read item dimensions
        item_dims = []
        for _ in range(n_items):
            tokens = lines[pos].split()
            pos += 1
            if len(tokens) == 2:
                w, h = map(int, tokens)
                item_dims.append((w, h))
            else:
                # Skip malformed lines
                continue

        instances.append(
            {
                "name": name,
                "n_items": n_items,
                "bin_dims": bin_dims,
                "item_dims": item_dims,
                "best_known": best_known,
            }
        )

    return instances


# ============================================================
#  MAIN EXPERIMENT SCRIPT
# ============================================================

def main():
    data_dir = "data"
    os.makedirs("plots/1D", exist_ok=True)
    os.makedirs("plots/2D", exist_ok=True)

    # -------------------------------
    # 1D: Beasley binpack1..8
    # -------------------------------
    instances_1d = []

    for i in range(1, 5):
        path = os.path.join(data_dir, f"binpack{i}.txt")
        sizes, C, best_known, name = load_orlib_1d(path, instance_index=0, float_items=False)
        instances_1d.append(
            {
                "name": name,
                "sizes": sizes,
                "capacity": C,
                "best_known": best_known,
                "float": False,
            }
        )

    for i in range(5, 9):
        path = os.path.join(data_dir, f"binpack{i}.txt")
        sizes, C, best_known, name = load_orlib_1d(path, instance_index=0, float_items=True)
        instances_1d.append(
            {
                "name": name,
                "sizes": sizes,
                "capacity": C,
                "best_known": best_known,
                "float": True,
            }
        )

    results_1d = []

    for inst in instances_1d:
        print(f"Solving 1D instance {inst['name']} ...")
        solver = ACOSingleDimBinPacking(
            inst["sizes"],
            inst["capacity"],
            num_ants=18,
            max_iters=90,
            alpha=1.0,
            beta=4.0,
            rho=0.1,
            Q=1.0,
            greedy_prob=0.15,
            seed=42,
        )

        best_bins, unused, runtime, conv = solver.run()
        gap = best_bins - inst["best_known"]

        results_1d.append(
            {
                "Instance": inst["name"],
                "n": len(inst["sizes"]),
                "Capacity": inst["capacity"],
                "Best Known": inst["best_known"],
                "Total Used Bins": best_bins,
                "Gap": gap,
                "Unused Capacity": round(unused, 2),
                "Runtime (s)": round(runtime, 2),
            }
        )

        conv_path = os.path.join("plots/1D", f"{inst['name']}_convergence.png")
        load_path = os.path.join("plots/1D", f"{inst['name']}_loads.png")
        solver.plot_convergence(conv_path)
        solver.plot_load_distribution(load_path)

    print("\n1D Results Table:")
    df1 = pd.DataFrame(results_1d)
    try:
        print(df1.to_markdown(index=False))
    except Exception:
        print(df1)

    # -------------------------------
    # 2D: area-based approximation
    # -------------------------------
    results_2d = []

    two_d_path = os.path.join(data_dir, "binpack2D.txt")
    if os.path.exists(two_d_path):
        instances_2d = load_orlib_2d(two_d_path)

        for inst in instances_2d:
            print(f"Solving 2D instance {inst['name']} (area approximation) ...")

            approx = AreaApprox2DBinPacking(
                inst["item_dims"],
                inst["bin_dims"],
                num_ants=15,
                max_iters=70,
                alpha=1.0,
                beta=3.0,
                rho=0.1,
                Q=1.0,
                seed=42,
            )

            best_bins, unused_area, runtime, conv = approx.run()

            if best_bins <= approx.num_bins_avail:
                used_bins = best_bins
                unused_report = round(unused_area, 1)
            else:
                used_bins = "Infeasible"
                unused_report = "N/A"

            results_2d.append(
                {
                    "Instance": inst["name"],
                    "n": inst["n_items"],
                    "Bins Available": approx.num_bins_avail,
                    "Total Used Bins": used_bins,
                    "Unused Area": unused_report,
                    "Runtime (s)": round(runtime, 1),
                }
            )

            conv_path = os.path.join("plots/2D", f"{inst['name']}_convergence.png")
            load_path = os.path.join("plots/2D", f"{inst['name']}_loads.png")
            approx.plot_convergence(conv_path)
            approx.plot_load_distribution(load_path)

        print("\n2D Results Table (Area Approximation):")
        df2 = pd.DataFrame(results_2d)
        try:
            print(df2.to_markdown(index=False))
        except Exception:
            print(df2)
    else:
        print("\nNo binpack2D.txt found in data/ – skipping 2D experiments.")



if __name__ == "__main__":
    main()


Solving 1D instance u120_00 ...
Solving 1D instance u250_00 ...
Solving 1D instance u500_00 ...
Solving 1D instance u1000_00 ...
Solving 1D instance t60_00 ...
Solving 1D instance t120_00 ...
Solving 1D instance t249_00 ...
Solving 1D instance t501_00 ...

1D Results Table:
| Instance   |    n |   Capacity |   Best Known |   Total Used Bins |   Gap |   Unused Capacity |   Runtime (s) |
|:-----------|-----:|-----------:|-------------:|------------------:|------:|------------------:|--------------:|
| u120_00    |  120 |        150 |           48 |                48 |     0 |               122 |         17.13 |
| u250_00    |  250 |        150 |           99 |               100 |     1 |               217 |         38.82 |
| u500_00    |  500 |        150 |          198 |               200 |     2 |               363 |        105.88 |
| u1000_00   | 1000 |        150 |          399 |               403 |     4 |               686 |        281.72 |
| t60_00     |   60 |        100 |       